### The concept

This article is a trap and you, dear reader, have fallen right in. I am not going to explain elliptic curves, at least not directly. I have a bias: I don't believe elliptic curves should be part of an _Introduction to blockchain cryptography_ series. They're certainly interesting, but they don't contain any important concepts. In principal, we could build an equally secure (but less efficient) blockchain without them.

The problem is that this **seems** wrong. You're here to learn about blockchain cryptography and if you don't know anything about elliptic curves, you probably feel like you're missing something crucial. There are lots of protocols and projects in this space that depend on ECDSA signatures, Pedersen commitments, secret sharing schemes and many more algorithms that all seem to rely on elliptic curves. In fact, I've heard multiple people refer to some version of [the world of elliptic curves](https://www.youtube.com/watch?v=_IP1xXlEVls&feature=youtu.be&t=578) as a shorthand for this whole industry. However, if you jump straight into performing calculations on elliptic curves there is a risk that you end up thinking: "sure, I can see what you're doing, but what does any of this have to do with cryptography?".

So the goal of this article is to develop some intuition on the role of elliptic curves, mostly by ignoring them, and then refer you to any of the many resources that dive into the details. [This one](https://blog.cloudflare.com/a-relatively-easy-to-understand-primer-on-elliptic-curve-cryptography/) strikes an excellent balance between clarity and completeness.

I'm presenting this article as a Jupyter notebook so that you can modify and experiment with the code. Please take the time to explore.

**WARNING:** There are likely readers who understand the subtleties better than me. If I've misunderstood or downplayed something, please comment.

We shall start by talking about strings through the lense of a common mathematical structure (specifically, _groups_ for those who know the term). This is going to be a long journey with a variety of topics, so to avoid getting lost I will place a marker here on our list of pending concepts (a _concept stack_ if you will).

In [ ]:
class ConceptStack():
    pending = []
    
    def new(self, concept):
        self.pending.append(concept)

    def show(self):
        print("PENDING CONCEPTS:\n{} <-- You are here".format("\n".join(self.pending)))
    
    def complete(self):
        print("ACHIEVEMENT UNLOCKED: you understand \"{}\"\n".format(self.pending.pop()))
        
concepts = ConceptStack()
concepts.new("Why are elliptic curves useful?") # we shall defer this until the end
concepts.new("Treating strings as a group")
concepts.show()

### Overloading operators

I was recently reminded that most developers are already familiar with the core concept under a different name: overloading operators. My goal is to leverage this observation so we can discuss mathematics on your (and my) home ground.

#### Adding strings

Consider the following python result:

In [ ]:
assert("abc" + "def" == "abcdef") 

It is a complete abuse of mathematical notation, but as long as you remember that addition means concatenation, you can add strings together.

#### Multiplying strings by numbers

What about multiplication?

In [ ]:
assert(3 * "abc" == "abcabcabc")

That actually follows pretty naturally, since multiplication is just repeated addition. How far can we take this?

#### Associativity

Adding strings also contains another useful property that mathematicians call _associativity_. You can perform addition (concatenation) operations in any order:

In [ ]:
a = "abc"
b = "def"
c = "ghi"
assert((a + b) + c == a + (b + c))

#### Commutativity

Note that this is not the same as providing _commutativity_

In [ ]:
a = "abc"
b = "def"
assert(a + b != b + a)

As it turns out, that doesn't actually matter for our purposes, but we'll fix it anyway in a moment.

#### Zero

In regular mathematics there is a special number - the number 0 - that has no effect when you add it to any other number

In [ ]:
x = 42
assert(x + 0 == x)
assert(0 + x == x)

When adding strings, this special _zero element_ is the empty string.

In [ ]:
ZERO = ""
x = "abc"

assert(x + ZERO == x)
assert(ZERO + x == x)

#### Negating elements

The negative of a number can be defined in terms of zero. When you add a number to its negative you get zero.

In [ ]:
x = 42
assert(-x + x == 0)
assert(x + (-x) == 0)

The equivalent concept when adding strings would be a value that can be added to the target string to produce the empty string. In other words, we want to implement the following function:

In [ ]:
ZERO = ""

def negative(string):
   raise NotImplementedError("How can you define this?")

x = "abc"
# print(negative(x)) # ???
# assert(x + negative(x) == ZERO)

And now we have a problem: there is no sensible way to define the `negative` function. There is no string that you can concatenate to a non-empty string in order to produce the empty string. Strings cannot be negated.

There is an analogous problem in the world of finite field (regular, non-elliptic-curve) cryptography, which will require a digression into _multiplicative groups_ to properly explain. Fortunately, that explanation will prove useful anyway so we might as well talk about it now.

In [ ]:
concepts.new("Groups and Multiplicative Groups")
concepts.show()

### Groups

We're working towards treating strings under a formalism that mathematicians call a _group_. If you haven't seen groups before, the resources written for mathematicians, like the [wikipedia page](https://en.wikipedia.org/wiki/Group_(mathematics)), are very intimidating but the concept is pretty simple: there is a short set of abstract rules and anything that complies with those rules is a group. Any result that assumes the abstract rules (and nothing else) automatically applies to all groups. It's a lot like an abstract class: any algorithm that operates on the abstract class automatically works on all instances.

Let's wrap our string concatenation properties in a class so we can look at the structure and then generalize it to all groups.

In [ ]:
class StringConcatenationGroupElement:
  @classmethod
  def ZERO(cls):
   return StringConcatenationGroupElement("")

  def __init__(self, _value):
    self.value = _value

  def add(self, otherElement):
    return StringConcatenationGroupElement(self.value + otherElement.value)

  def negative(self):
    raise NotImplementedError("How can you define this?")

The properties that make this a group are (not coincidentally) the same ones we've already seen:

1. you can always `add` two group elements and it will produce a group element
2. the addition is associative (defined above)
3. there is a `ZERO` element that has no effect when added to another element
4. every element can be negated (defined above)

Any set of elements that conform to these rules form a group.

#### Multiplicative Groups

As we've already seen the `add` operation doesn't necessarily have anything to do with the common understanding of addition. In fact, it is often implemented as the multiplication of two numbers. As you might imagine, this can get confusing.

The elegant way to solve this is to generalize the terminology so it doesn't suggest anything about the actual implementation:
* Instead of _adding_ two elements, you _perform the group operation_ whatever that is (add, multiply, concatenate, etc)
* The _zero_ element is renamed to the _identity_ element
* The _negative_ operation is renamed to the _inverse_ operation

In practice, most people just choose a symbol to represent the group operation and then adjust their vocabulary accordingly:

| _ | Generic Group | Additive Group | Multiplicative Group |
|---|---------------|----------------|----------------------|
**Operation** | Group operation | Addition | Multiplication |
**Operator Symbol** |· | + | * |
**Identity symbol** | IDENTITY or id | ZERO or 0 | ONE or 1 |
**Identity definition** | a·id == a <br> id.a == a | a + 0 == a <br> 0 + a == a | a * 1 == a <br> 1 * a == a |
**Inverse of a** | a<sup>-1</sup> | -a | a<sup>-1</sup> |
**Inverse definition** | a·a<sup>-1</sup> == id <br> a<sup>-1</sup>·a == id | a + (-a) == 0 <br> -a + a == 0 |  a \* a<sup>-1</sup> == 1 <br> a<sup>-1</sup> \* a == 1

If it helps, you can think of this as class inheritance.

In [ ]:
"""
The structure of an abstract group.
Any implementation should preserve the following properties:
- Let a, b and c be any group elements
     assert(type(a) is GroupElement)
     assert(type(b) is GroupElement)
     assert(type(c) is GroupElement)
- I = GroupElement.IDENTITY()
- assert(type(I) is GroupElement)
- assert(type(a._op(b)) is GroupElement)
- assert(a._op(I) == a)
- assert(I._op(a) == a)
- assert(type(a._inverse()) is GroupElement)
- assert(a._inverse()._op(a) == I)
- assert(a._op(a._inverse()) == I)
- assert(a._op(b)._op(c) == a._op(b._op(c)))
"""
class GroupElement:
  @classmethod
  def IDENTITY(cls):
    raise NotImplementedError("Should be overriden")

  def __init__(self, _value):
    self.value = _value

  def _op(self, otherElement):
    raise NotImplementedError("Should be overriden")

  def _inverse(self):
    raise NotImplementedError("Should be overriden")

"""
The structure of an abstract multiplicative group
This is a generic group with some notational conventions:
- we refer to the identity as 'ONE'
- we refer to the operation as 'multiplication'
"""
class MultiplicativeGroupElement(GroupElement):
   @classmethod
   def ONE(cls):
     return cls.IDENTITY()

   def mul(self, otherElement):
     return self._op(otherElement)

   def inverse(self):
     return self._inverse();

"""
The structure of an abstract additive group
This is a generic group with some notational conventions:
- we refer to the identity as 'ZERO'
- we refer to the operation as 'addition'
- we refer to the inverse element as 'negative'
"""
class AdditiveGroupElement(GroupElement):
   @classmethod
   def ZERO(cls):
     return cls.IDENTITY()

   def add(self, otherElement):
     return self._op(otherElement)

   def negative(self):
     return self._inverse()

"""
Our String Concatenation group is an additive group
- the group operation (addition) is string concatenation
- the identity (ZERO) is the empty string
- the inverse (negative) element is not yet determined
"""
class StringConcatenationGroupElement(AdditiveGroupElement):
   @classmethod
   def IDENTITY(cls):
     return StringConcatenationGroupElement("")

   def _op(self, otherElement):
     return StringConcatenationGroupElement(self.value + otherElement.value)

   def _inverse(self):
     raise NotImplementedError("How can you define this?")

The crucial point is that at a theoretical level, it doesn't matter how an element is represented, or how elements are combined, or which symbol is used to denote the combination. If the elements and the combining operation form a group, they can be transformed into equivalent groups in other representations. You may have guessed the punchline: if there are algorithms (important, industry-defining algorithms) that only require the group operation, they can be implemented equally well using any representation. All of the magic is in the algorithm itself, not in the representation.

In theory there is no difference between theory and practice, but in practice there is. Of course the representations do actually matter or why would anyone bother changing them? We will investigate the differences in due time but first we should tie up some loose ends.

Now that we understand multiplicative groups, let's look an example used throughout cryptography.

In [ ]:
concepts.complete()
concepts.new("A common group: integers modulo a prime")
concepts.show()

### Integers modulo a prime

There is a very common way to construct a group:

1. Pick a prime number `p`
2. The group elements are all positive integers less than `p`
3. The group operation is multiplication modulo `p`

This is a multiplicative group because the group operation is multiplication. Just to be explicit, this means that you can multiply the elements together (and reduce them modulo `p`), but addition or subtraction is not defined. Exponentiation is allowed, because that's just a shorthand description of repeated multiplication.

For example, if `p` is 5, the multiplication table for this group is:

    
| * | 1 | 2 | 3 | 4 |
|---|---|---|---|---|
**1**| 1 | 2 | 3 | 4 
**2** | 2 | 4 | 1 | 3
**3** | 3 | 1 | 4 | 2 
**4** | 4 | 3 | 2 | 1 

**Challenge:**  Prove to yourself that these elements form a group. If you're feeling daring, prove that the procedure will always produce a group.

We will need this group in a moment but for now I'd just like to draw a lesson: positive integers under multiplication are almost a group. They are associative [ `(a * b) * c = a * (b * c)` ] and have an identity [ the number 1 ] but they do not have inverses, since multiplying positive integers never makes them smaller. This is analogous to string concatenation, which never makes strings smaller. However, the modulo function normalizes the integers so they can be treated as equivalent to smaller integers. This makes it possible for the product of two integers larger than 1 (like 2 and 3 in this example) to be 1. Those integers are inverses of each other in this group.

In [ ]:
p = 5
assert(2 * 3 % p == 1) # 2 and 3 are inverses mod 5

This suggests a strategy to complete our string concatenation group: normalize the elements so concatenating non-empty strings can produce the empty string.

In [ ]:
concepts.complete()
concepts.show()

### Strings as an additive group

We need a mechanism to normalize strings. The one I have chosen is:

* remove any characters that are not lowercase alphabets
* sort the characters in alphabetical order. As a bonus, this introduces commutativity (defined above)
* any three identical characters are replaced with the next character in the alphabet (_aaa_ becomes _b_, _bbb_ becomes _c_ ...)
* _zzz_ is replaced with the empty string

Note that this gives every string an inverse, which completes our group. In code, this looks like:

In [ ]:
from string import ascii_lowercase
from functools import reduce

"""
Our String Concatenation group is an additive group
- the group operation (addition) is string concatenation with normalization
- the identity (ZERO) is the empty string
- the inverse (negative) element can be added to the original element to produce ZERO
"""
class StringConcatenationGroupElement(AdditiveGroupElement):
   @classmethod
   def IDENTITY(cls):
     return StringConcatenationGroupElement("")

   # Use a complete mapping of letter counts internally for algorithmic simplicity
   # The string "aac" is represented by {"a": 2, "b": 0, "c": 1, ... }
   @classmethod
   def _normalize(cls, stringValue):
     tally = {c: 0 for c in ascii_lowercase}
     lowerAlphabets = filter(lambda c: c.islower(), stringValue)
     for c in lowerAlphabets:
       tally[c] += 1
     for c in ascii_lowercase[:-1]:
       nextChar = chr(ord(c) + 1)
       tally[nextChar] += int(tally[c] / 3)
       tally[c] %= 3
     tally['z'] %= 3
     return tally

   # A convenience function - multiplication is just repeated addition
   # In a production system, you would use the double-and-add method, which is far more efficient
   def mul(self, m):
     return reduce(lambda result, i: result.add(self), range(m), StringConcatenationGroupElement.ZERO())

   def __init__(self, _value):
     self.value = StringConcatenationGroupElement._normalize(_value)

   def _op(self, otherElement):
     return StringConcatenationGroupElement(str(self) + str(otherElement))

   def _inverse(self):
     if len(str(self)) == 0:
       return StringConcatenationGroupElement.ZERO()
     firstChar = str(self)[0]
     firstCharIdx = ord(firstChar) - ord('a') # the index in the alphabet of the first character
     inverseStr = reduce(
       lambda inv, c: inv + (2 - self.value[c]) * c, # we need 2 instances of 'c' shared between both strings (the third is carried from the previous character)
       ascii_lowercase[firstCharIdx + 1:], # cycle through every character after the first one
       (3 - self.value[firstChar]) * firstChar # we need 3 instances of 'firstChar' shared between 'value' and its inverse
     )
     return StringConcatenationGroupElement(inverseStr)

   # convert the internal representation (a tally of letters) to the equivalent string
   def __str__(self):
     return reduce(lambda string, c: string + self.value[c] * c, ascii_lowercase, "")

If this seems like an arbitrary and contrived way to form a group - it absolutely is. In my defence, it is significantly more straightforward than the actual method that cryptographers use to form elliptic curve groups.

<hr>

###### A skippable digression

The purpose of this article is to reassure you that this is not essential understanding, but for comparison, the procedure that Elizabeth Curve, the inventor of elliptic curves<sup>[citation needed]</sup> came up with is:
* draw a crazy-looking curve
* start by pretending that you will treat the points on the curve as group elements
* invent a completely different point that doesn't exist on the curve and call it the Point at Infinity. This is the identity element.
* describe the group operation as _addition_. This means we need a mechanism to _add_ two points together. Specifically:
   * draw a line between the points (or use the tangent if both elements are the same point)
   * find the third point where the line intersects the curve
   * reflect that point over the x axis
   * the final point that you land on is the sum of the original inputs
   * solve all edge cases by appealing to the Point at Infinity in a way that ensures the consistency of the group
* take a moment to admire the equations you've come up with and then completely forget that they had anything to do with curves or lines or tangents
* reframe any _division_ operation that you might have written as an _inverse multiplication_ operation so it will work with arbitrary field elements (a _field_ is another mathematical structure, like a group but with two operations and their inverses. The _Real_ everyday numbers that you already know how to add, subtract, multiply and divide are an example of a field).
* treat these equations as if they apply to integers modulo some prime (another field), destroying any mental picture you might have had that gave you an intuitive understanding of what you were doing
* you are left with:
    * a way to describe a group element as a pair of integer coordinates. This includes a special Point at Infinity.
    * a set of restrictions for which pairs of integer coordinates are valid
    * a mathematical rule for adding two valid elements together to produce another valid element
* as it turns out, if you choose the parameters carefully, this process produces a group - go forth and add
<hr>

The last building block we will need is the relationship between different groups.

In [ ]:
concepts.complete()
concepts.new("Translating between groups")
concepts.show()

### Cyclic Groups

All groups follow the same set of rules (by definition), but they may differ in internal structure ie. the specific relationships between the group elements. One way to standardize the structure is to pick a particular element (typically called `g` for generator) and then apply the group operation repeatedly until you reach the identity (assuming the group is finite). These elements form a cyclic group.

For example, consider the multiplicative group of integers mod 11. The group elements are the positive integers less than 11. However, if we choose the generator 4 we can generate the cyclic group:
* 4 mod 11 ≡ 4
* 4<sup>2</sup> mod 11 ≡ 5
* 4<sup>3</sup> mod 11 ≡ 9
* 4<sup>4</sup> mod 11 ≡ 3
* 4<sup>5</sup> mod 11 ≡ 1

For powers of 6 or higher we will just cycle through these values again. These 5 elements form a smaller group within the group (a subgroup). The multiplication table for this group is:

 * | 4 | 5 | 9 | 3 | 1 
---|---|---|---|---|---
**4**| 5 | 9 | 3 | 1 | 4 
**5** | 9 | 3 | 1 | 4 | 5
**9** | 3 | 1 | 4 | 5 | 9
**3** | 1 | 4 | 5 | 9 | 3
**1** | 4 | 5 | 9 | 3 | 1

Notice that all rows and columns are shifted versions of each other. Each step right or down is equivalent to multiplying by 4 (mod 11). Abstractly, we could think of the elements in terms of their relationship to the generator. We could also abstract away whether we're using a multiplicative group or an additive group, since the same logic would apply in either case.

<pre><table><tr>
<td><table>
<tr><td><b>*</b></td><td><b>g</b></td><td><b>g<sup>2</sup></b></td><td><b>g<sup>3</sup></b></td><td><b>g<sup>4</sup></b></td><td><b>1</b></td></tr>
<tr><td><b>g</b></td><td>g<sup>2</sup></td><td>g<sup>3</sup></td><td>g<sup>4</sup></td><td>1</td><td>g</td></tr>
<tr><td><b>g<sup>2</sup></b></td><td>g<sup>3</sup></td><td>g<sup>4</sup></td><td>1</td><td>g</td><td>g<sup>2</sup></td></tr>
<tr><td><b>g<sup>3</sup></b></td><td>g<sup>4</sup></td><td>1</td><td>g</td><td>g<sup>2</sup></td><td>g<sup>3</sup></td></tr>
<tr><td><b>g<sup>4</sup></b></td><td>1</td><td>g</td><td>g<sup>2</sup></td><td>g<sup>3</sup></td><td>g<sup>4</sup></td></tr>
<tr><td><b>1</b></td><td>g</td><td>g<sup>2</sup></td><td>g<sup>3</sup></td><td>g<sup>4</sup></td><td>1</td></tr>
</table></td>

<td><table>
<tr><td><b>+</b></td><td><b>g</b></td><td><b>2g</b></td><td><b>3g</b></td><td><b>4g</b></td><td><b>0</b></td></tr>
<tr><td><b>g</b></td><td>2g</td><td>3g</td><td>4g</td><td>0</td><td>g</td></tr>
<tr><td><b>2g</b></td><td>3g</td><td>4g</td><td>0</td><td>g</td><td>2g</td></tr>
<tr><td><b>3g</b></td><td>4g</td><td>0</td><td>g</td><td>2g</td><td>3g</td></tr>
<tr><td><b>4g</b></td><td>0</td><td>g</td><td>2g</td><td>3g</td><td>4g</td></tr>
<tr><td><b>0</b></td><td>g</td><td>2g</td><td>3g</td><td>4g</td><td>0</td></tr>
</table></td>
</tr></table></pre>

Since all cyclic groups will conform to this structure, and this structure is only thing that can vary between groups, **all cyclic groups of the same size are equivalent**.

### Translating a cryptographic attack

This is where it gets (more?) fun. We have come so far so we might as well go a little further. We are going to combine all of these concepts to view a real world cryptographic attack under the light of group equivalences. Specifically, we will develop the attack for three groups simultaneously so we can compare the implementations. The particular numbers don't matter - just notice the high level pattern and similarities.

#### The Setup

Pick a cyclic group.

**WARNING:** For simplicity of explanation, the particular groups that we are comparing do not have the same sizes so they're not actually equivalent. That won't matter for the core of this dicussion but it is a minor correction that should be taken into account.

###### Finite Field Cryptography
* Choose a large prime `p` and generator `g`. The group elements are the powers of `g` mod `p` (any number that can be expressed as <code>g<sup>a</sup> mod p</code> with an integer `a`)
* For simplicity, we won't write mod `p` in every equation - it is always implied.

In [ ]:
class IntegersModPGroupElement(MultiplicativeGroupElement):
    p = 32452843
    
    @classmethod
    def IDENTITY(cls):
      return IntegersModPGroupElement(1)

    # A convenience function - exponentiation is just repeated multiplication
    def exp(self, exponent):
      return IntegersModPGroupElement(pow(self.value, exponent, self.p))

    def __init__(self, _value):
      self.value = _value % self.p
    
    def _op(self, otherElement):
      return IntegersModPGroupElement(self.value * otherElement.value)

    def _inverse(self):
       return self.exp(self.p-2) # challenge: convince yourself that this produces an inverse

# The generator of the Finite Field Cryptography group
FFC_g = IntegersModPGroupElement(42)

###### String Concatenation Cryptography
* Choose a generator `G`
* The group elements are the multiples of `G` (any string that can be expressed as `a·G` with an integer `a`, where multiplying is taken to mean repeatedly _adding_ `G` to itself using our special definition for adding strings and normalization is always implied).

In [ ]:
# The generator of the String Concatenation Cryptography group
SCC_G = StringConcatenationGroupElement("generator")

###### Elliptic Curve Cryptography
* Choose an elliptic curve over a finite field (whatever that means) and a generator `G`. We will use [secp256k1](https://en.bitcoin.it/wiki/Secp256k1)
* The group elements are the multiples of `G` (any point that can be expressed as `a·G` with an integer `a`, where multiplying is taken to mean repeatedly _adding_ `G` to itself using a special definition for adding points and normalization is always implied).

In [ ]:
# We won't implement this class
class Secp256k1GroupElement(AdditiveGroupElement):
    pass

# Together, these two values represent a point on the elliptic curve.
ECC_G_x = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
ECC_G_y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
# That point corresponds to a single group element
ECC_G = Secp256k1GroupElement((ECC_G_x, ECC_G_y))

#### Generating a keypair

Choose a random 6-digit integer as a private key `d`. For multiplicative groups, our public key is <code>D = g<sup>d</sup></code>. For additive groups, it is <code>D = d·G</code>. Note that both of these equations are just shorthand for "perform the group operation with the generator `d` times"

In [ ]:
# Use the same private key for all groups
d = 726960
FFC_D = FFC_g.exp(d)
SCC_D = SCC_G.mul(d) # This may take a few seconds to calculate

# We did not implement Secp256k1GroupElement 
# so we cannot calculate the elliptic curve public key
# ECC_D = ECC_G.mul(d)
# For reference, this is what is should be.
# Note that the group element is a point on the curve
ECC_D = Secp256k1GroupElement((
  0x97f9735766a16f4eef5965ce6acd443b27e8d10f204987738f9978f588120c05,
  0xa0b90b99a85a37f77cfc14a7c26e14b338cdab828b33002d5ad3d71353560bd7
)) 

#### Attacking the keypair

Let's pretend we're an attacker who doesn't know the private key `d`. How could we figure it out from `D`? We assume that in all our groups it is easy to derive the public key from the private key but hard to go backwards.

The obvious answer is to try all possible `d` values until we find the right one, but we can do better than that. 

Let's split `d` into two 3-digit numbers: <code>d = d<sub>0</sub> || d<sub>1</sub></code>. Mathematically, this is equivalent to writing <code>d = 1000·d<sub>0</sub> + d<sub>1</sub></code>.

This lets us break the public key into two pieces that each depend on only half of the private key.

###### Multiplicative Groups

D = g<sup>d</sup><br>
D = g<sup>1000·d<sub>0</sub> + d<sub>1</sub></sup><br>
D = g<sup>1000·d<sub>0</sub></sup> · g<sup>d<sub>1</sub></sup><br>
g<sup>-1000·d<sub>0</sub></sup> · D = g<sup>d<sub>1</sub></sup><br>
((g<sup>-1</sup>)<sup>1000</sup>)<sup>d<sub>0</sub></sup> · D = g<sup>d<sub>1</sub></sup><br>

###### Additive Groups
D = d·G<br>
D = (1000·d<sub>0</sub> + d<sub>1</sub>)·G<br>
D = 1000·d<sub>0</sub>·G + d<sub>1</sub>·G<br>
-1000·d<sub>0</sub>·G + D = d<sub>1</sub>·G<br>
d<sub>0</sub>·(1000·(-G)) + D = d<sub>1</sub>·G<br>

Once again, note that these are two different notations for the same set of group operations.

We can now execute a [meet-in-the-middle attack](https://en.wikipedia.org/wiki/Meet-in-the-middle_attack):
* Calculate and save all possible values for the right hand side of the equation (the part that depends on <code>d<sub>1</sub></code>)
* Calculate and save all possible values for the left hand side of the equation (the part that depends on <code>d<sub>0</sub></code>)
* Find a value that exists on both lists. This occurs for the correct values of <code>d<sub>0</sub></code> and <code>d<sub>1</sub></code>

In [ ]:
## The FFC attack ##

# store all possible d1 values for the right-hand-side, indexed by g^d1
saved_RHS = {}
RHS = IntegersModPGroupElement.IDENTITY()
for d1 in range(1000):
    # at this point RHS = FFC_g.exp(d1)
    saved_RHS[RHS.value] =  d1
    # use each RHS value to calculate the next one so each step is only one group operation
    RHS = RHS.mul(FFC_g)
    
# compare this to the possible values for the left-hand-side
base = FFC_g.inverse().exp(1000)
LHS = FFC_D
for d0 in range(1000):
    # at the point LHS = base.exp(d0).mul(FFC_D)
    if(LHS.value in saved_RHS):
        computed_d = 1000 * d0 + saved_RHS[LHS.value]
    # use each LHS value to calculate the next one so each step is only one group operation
    LHS = base.mul(LHS)

assert(computed_d == d)

In [ ]:
## The SSC attack ##

# store all possible d1 values for the right-hand-side, indexed by d1·G
saved_RHS = {}
RHS = StringConcatenationGroupElement.IDENTITY()
for d1 in range(1000):
    # at this point RHS = FFC_g.mul(d1)
    saved_RHS[str(RHS)] =  d1
    # use each RHS value to calculate the next one so each step is only one group operation
    RHS = RHS.add(SCC_G)
    
# compare this to the possible values for the left-hand-side
point = SCC_G.negative().mul(1000)
LHS = SCC_D
for d0 in range(1000):
    # at the point LHS = point.mul(d0).add(FFC_D)
    if(LHS in saved_RHS):
        computed_d = 1000 * d0 + saved_RHS[LHS]
    # use each LHS value to calculate the next one so each step is only one group operation
    LHS = point.add(LHS)

assert(computed_d == d)

The first thing to notice about this attack is that it's very powerful: instead of exhausting over the possible values for the key, you can exhaust over each half separately, which is significantly faster. The existence of this attack means that the private key is only as strong as half its length.

More importantly for this article, the attack is almost identical in the group of integers modulo `p` and the string concatenation group. The equations are the same, they just need to be translated between the multiplicative and additive representations. The implementations are very different (multiplying by integers modulo `p` is nothing like concatenating strings and normalizing them) but since the equations are the same it is trivial to translate the algorithm between the representations. Naturally, if we had implemented the elliptic curve Secp256k1 group, we could translate the attack into that context just as easily.

In fact, this is the source of many elliptic curve algorithms:
* If you know the [Diffie Hellman key exchange](https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange) algorithm, replace the integers modulo `p` with an elliptic curve group and you have [Elliptic Curve Diffie Hellman](https://en.wikipedia.org/wiki/Elliptic-curve_Diffie%E2%80%93Hellman)
* If you know [DSA signatures](https://en.wikipedia.org/wiki/Digital_Signature_Algorithm), you can easily derive [ECDSA](https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm)
* If you know [ElGamal encryption](https://en.wikipedia.org/wiki/ElGamal_encryption) then you have [EEECC](https://cse.unl.edu/~ssamal/crypto/EEECC.pdf)

The implementations look far more complicated but the algorithms are the same. If you wanted to, you could implement these algorithms using the newly invented field of String Concatenation Cryptography. Under the hood, it's all just group operations.  Recall, if you're interested in actually constructing elliptic curve groups, I recommend [this article](https://blog.cloudflare.com/a-relatively-easy-to-understand-primer-on-elliptic-curve-cryptography/). But if you're trying to understand a particular algorithm, _the fact that it uses elliptic curves is almost incidental_. Feel free to think of every elliptic curve addition as simply "doing the group operation".

This leads to our final topic: why does anyone bother using elliptic curves at all?

In [ ]:
concepts.complete()
concepts.show()

### Why are elliptic curves useful?

If translating between groups is trivial, why would any particular representation be preferable? 

Asked another way, now that we have invented String Concatenation Cryptography, should you actually use it? The intuitive (and correct) answer is: of course not!! This is because there is more structure than strictly required, and this can be exploited. We insist that for legitimate uses of the group only the concatenation operation is defined, but that doesn't prevent attackers from using compression, or substrings, or noticing the similarities in the calculation of `10 * "ab"` and `10 * "cd"`. Every possible manipulation of the elements is a potential avenue to short-circuit the official group operation, and additional structure increases the attack surface.

There is a similar fact about the group of integers modulo a prime. It is possible for an attacker to step outside of the group and attempt to recover a private key using the [General Number Field Sieve](https://en.wikipedia.org/wiki/General_number_field_sieve), which runs in sub-exponential time. The elliptic curve groups, however, are not susceptible to this algorithm. Their inscrutability is their advantage, because they have minimal unnecessary structure. Since the security of a system is limited by the best possible attack, this means that elliptic curve groups can achieve the same security with smaller key sizes than would be required in the equivalent integers-modulo-a-prime group.

This leads to a rather banal conclusion: the reason they are used is that for a given level of security, the elliptic curve counterparts of cryptographic algorithms are faster.